<a href="https://colab.research.google.com/github/corentinBlanchard/chatbot-IA/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
#Imports pour recuperation de fichiers a partir de google drive ( ou la DB est stockee)
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [15]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [53]:
#Importation des donnees sur la conversation concernant l'IA
ia_db = drive.CreateFile({'id':'1asz0LQpmjqAfx7_rc6WxkOeYOS848F2S'})
ia_db.GetContentFile('ia.yml') 

botprofile_db = drive.CreateFile({'id':'1Uuz-UJTelqUrndGa1xFoB9YUbgZckzb-'})
botprofile_db.GetContentFile('botprofile.yml') 

computers_db = drive.CreateFile({'id':'13CsF8ldiPjjwLlp317rnXFy3xs6DX8Bd'})
computers_db.GetContentFile('computers.yml')

emotion_db = drive.CreateFile({'id':'1zz3HfhaQvnzTmGRk4Ax7L61RNIutnBjS'})
emotion_db.GetContentFile('emotion.yml')

food_db = drive.CreateFile({'id':'10cTQChOe9zzsGBJyYiCAp-KBk2rYCkTE'})
food_db.GetContentFile('food.yml')

gossip_db =  drive.CreateFile({'id':'1Rt5to2LxH69v_qcDiYAJsVWjhn_E1c1A'})
gossip_db.GetContentFile('gossip.yml')

greetings_db =  drive.CreateFile({'id':'1rE2IBzv8JWA-GwAiEaYKUArP13DFyvcK'})
greetings_db.GetContentFile('greetings.yml')

health_db =  drive.CreateFile({'id':'1Gkg_LJBiRUk10-b9a5npJtMn6UNuEY6g'})
health_db.GetContentFile('health.yml')

history_db =  drive.CreateFile({'id':'1bKsUE4iGVqwADmqIHwPX3u3rogXs7dcD'})
history_db.GetContentFile('history.yml')

humor_db =  drive.CreateFile({'id':'1RUcaerZy4Se2zy9guQgAAVA_0N35pO8b'})
humor_db.GetContentFile('humor.yml')

literature_db =drive.CreateFile({'id':'1TdGULLQEXpiQvFwmB00V21qhfE9LGnq7'})
literature_db.GetContentFile('literature.yml')

money_db =drive.CreateFile({'id':'1mW98HXxply4YH8wha9BZg1tr09hrlX3W'})
money_db.GetContentFile('money.yml')

movies_db =drive.CreateFile({'id':'1V3WOQDZmUnFN-ZhG9nXYq4g3LhGMmjck'})
movies_db.GetContentFile('movies.yml')

politics_db =drive.CreateFile({'id':'1IgVS9x_7r4btfW_LyDg-py7sucRFduKp'})
politics_db.GetContentFile('politics.yml')

psychology_db =drive.CreateFile({'id':'1RApXTqebauwsMVqOCzC5aO_nB4nsg0NW'})
psychology_db.GetContentFile('psychology.yml')

science_db =drive.CreateFile({'id':'1AtIFu5F30pxNO-WzOnLNK7aKU7B4lgdM'})
science_db.GetContentFile('science.yml')

sports_db =drive.CreateFile({'id':'1OsBjPRq9VOCuVl-XSJIp0rTSC0zeJZv-'})
sports_db.GetContentFile('sports.yml')

trivia_db =drive.CreateFile({'id':'1vTwCh5WJNjEKnFGqRZfsRx7QhzXbkMcJ'})
trivia_db.GetContentFile('trivia.yml')


In [77]:
import yaml
doc_list = []
file = open('ia.yml')
docs = yaml.safe_load(file)
doc_list.append(docs)

file = open('botprofile.yml')
docs = yaml.safe_load(file)
doc_list.append(docs)

file = open('computers.yml')
docs = yaml.safe_load(file)
#doc_list.append(docs)

file = open('emotion.yml')
docs = yaml.safe_load(file)
#doc_list.append(docs)

file = open('food.yml')
docs = yaml.safe_load(file)
doc_list.append(docs)

file = open('gossip.yml')
docs = yaml.safe_load(file)
doc_list.append(docs)

file = open('greetings.yml')
docs = yaml.safe_load(file)
doc_list.append(docs)

file = open('health.yml')
docs = yaml.safe_load(file)
#doc_list.append(docs)

file = open('history.yml')
docs = yaml.safe_load(file)
doc_list.append(docs)

file = open('humor.yml')
docs = yaml.safe_load(file)
doc_list.append(docs)

file = open('literature.yml')
docs = yaml.safe_load(file)
doc_list.append(docs)

file = open('money.yml')
docs = yaml.safe_load(file)
doc_list.append(docs)

file = open('movies.yml')
docs = yaml.safe_load(file)
doc_list.append(docs)

file = open('politics.yml')
docs = yaml.safe_load(file)
doc_list.append(docs)

file = open('psychology.yml')
docs = yaml.safe_load(file)
doc_list.append(docs)

file = open('science.yml')
docs = yaml.safe_load(file)
doc_list.append(docs)

file = open('sports.yml')
docs = yaml.safe_load(file)
doc_list.append(docs)

file = open('trivia.yml')
docs = yaml.safe_load(file)
doc_list.append(docs)



In [78]:
#Create 1 table for questions and 1 table for answers
#TODO : import the rest of DB function of len of multiple answers
question_table = []
answers_table = []

for doc in doc_list:
  for sentences in doc['conversations']:
    question_table.append(sentences[0])
    answers_table.append(sentences[1])

print(len(question_table))

507


In [79]:
#preprocessing : nous decoupons notre BDD en mots en supprimant la ponctuation afin de créer un dictionnaire
words = []
#recuperation des mots 
for sentences in question_table :
  question = sentences.split()
  #answer = sentences[1].split()
  for word in question:
    words.append(word.lower())
  #for word in answer :
  #  words.append(word.lower())

In [80]:
#suppression de la ponctuation
import string
import re
re_punc = re.compile('[%s]' % re.escape(string.punctuation))
stripped = [re_punc.sub('', w) for w in words]

In [81]:
#Importation des stopwords
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words=stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [82]:
#Suppression des stopwords dans notre BDD

import string
import re
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))
#words = [w for w in stripped if not w in stop_words]
words = stripped

In [83]:
#Stemming pour réduction des mots à leur base
from nltk.stem.porter import PorterStemmer
# stemming of words
porter = PorterStemmer()
words = [porter.stem(word) for word in words]

In [87]:
# Bag of Words

from sklearn.feature_extraction.text import CountVectorizer
# create the transform
# vectorizer = TfidfVectorizer()
vectorizer = CountVectorizer()
# tokenize and build vocab
vectorizer.fit(words)
# encode document
vector = vectorizer.transform(words)
# summarize encoded vector
print("vector size : ")
print(vector.size)

vector size : 
1948


In [63]:
# Create table with questions and preprocessing them

questions_text = []
questions_bow = []


for sentences in question_table :
  question = sentences.split()
  table = []
  merged_data = []
  
  for word in question:
    table.append(word.lower())
  
  table = [re_punc.sub('', w) for w in table]
  #table = [w for w in table if not w in stop_words]
  table = [porter.stem(word) for word in table]
  merged_data.append(' '.join(table))

  vector = vectorizer.transform(merged_data)
  #print(vector.toarray())

  questions_text.append(merged_data)
  questions_bow.append(vector)

questions_bow_2=[]

for element in questions_bow:
  questions_bow_2.append(element.toarray())

[['what is ai'], ['what is ai'], ['are you sentient'], ['are you sentient'], ['are you sentient'], ['are you sapient'], ['are you sapient'], ['are you sapient'], ['are you sapient'], ['what languag are you written in'], ['what languag are you written in'], ['you sound like data'], ['you sound like data'], ['you are an artifici linguist entiti'], ['you are an artifici linguist entiti'], ['you are not immort'], ['you are not immort'], ['you are not immort'], ['you are not make sens'], ['you are not make sens'], ['you are not make sens'], ['you are not make sens'], ['you are not make sens'], ['you are immort'], ['you are immort'], ['you are immort'], ['you do not make ani sens'], ['you can not clone'], ['you can not clone'], ['you can not move'], ['you can not move'], ['bend over'], ['bend over'], ['robot laugh'], ['robot should die'], ['robot'], ['robot are stupid'], ['robot are not allow to lie'], ['robot are not allow to lie'], ['robot are not allow to lie'], ['robot'], ['it is a compu

In [64]:
#Convert user question to vector using the Bag of Words

def convert_question_user(question_user):
  question = question_user.split()
  table_words_user = []
  table_question_user = []


  for word in question:
    table_words_user.append(word.lower())

  table_words_user = [re_punc.sub('', w) for w in table_words_user]
  table_words_user = [porter.stem(word) for word in table_words_user]

  for element in table_words_user:
    if element=="":
      table_words_user.remove(element)

  table_question_user.append(' '.join(table_words_user))
  vector_question_user = vectorizer.transform(table_question_user).toarray()
  return vector_question_user



In [75]:
#Find the cosine distance to find the most similar sentence

import numpy as np
from numpy import dot
from numpy.linalg import norm
import random

# cosine similarity
def cosine(v1, v2):
    if norm(v1) > 0 and norm(v2) > 0:
        return dot(v1, v2) / (norm(v1) * norm(v2))
    else:
        return 0.0

def spacy_closest(token_list, vec_to_check):
  cosine_max = 0.0
  i_elements = []
  i=0
  for element in token_list:
    cos = cosine(element, vec_to_check)
    if float(cos) > cosine_max:
      cosine_max = cos
      i_elements = []
      i_elements.append(i)
    if float(cos)==cosine_max:
      i_elements.append(i)
  if cosine_max < 0.2 :
    return -1
    i = i+1
  return random.choice(i_elements)

In [73]:
def conversation(question):
  vector_question_user = convert_question_user(question_user=question)
  i = spacy_closest(questions_bow_2,vector_question_user[0])
  if i == -1 :
    return "I don't understand your question, please reformulate"
  return answers_table[i]

In [88]:
#Final block to speak with the Chatbot
while(True):
  question_user = input("Please type your question here : \n")
  if question_user == "exit":
    print("End of conversation")
    break
  print(conversation(question_user))

Please type your question here : 
exit
End of conversation
